### Gerar token do projeto

In [1]:
from project_lib import Project
project = Project(spark.sparkContext, '700141ff-aa85-452d-a8de-39d74e7de78b', 'p-1736ecb575fe8c3d4004b6ea585ac2644e5197d5')
pc = project.project_context

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20210527215457-0013
KERNEL_ID = 3456d2c7-ce99-40cf-920e-afb0b98a2269


### Importar bibliotecas

In [2]:
import sys
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3
import ibmos2spark, os
from pyspark.sql import SparkSession
import os, ibm_db_dbi as dbi
import ibm_db

### Creadenciais DB2

In [3]:
DB2_Hackathon_cert = os.path.join(os.path.expanduser('~'),'DB2_Hackathon_ssl.cert')
f = open(DB2_Hackathon_cert, "w")
f.write("""LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSURFakNDQWZxZ0F3SUJBZ0lKQVA1S0R3ZTNCTkxiTUEwR0NTcUdTSWIzRFFFQkN3VUFNQjR4SERBYUJnTlYKQkFNTUUwbENUU0JEYkc5MVpDQkVZWFJoWW1GelpYTXdIaGNOTWpBd01qSTVNRFF5TVRBeVdoY05NekF3TWpJMgpNRFF5TVRBeVdqQWVNUnd3R2dZRFZRUUREQk5KUWswZ1EyeHZkV1FnUkdGMFlXSmhjMlZ6TUlJQklqQU5CZ2txCmhraUc5dzBCQVFFRkFBT0NBUThBTUlJQkNnS0NBUUVBdXUvbitpWW9xdkdGNU8xSGpEalpsK25iYjE4UkR4ZGwKTzRUL3FoUGMxMTREY1FUK0plRXdhdG13aGljTGxaQnF2QWFMb1hrbmhqSVFOMG01L0x5YzdBY291VXNmSGR0QwpDVGcrSUsxbjBrdDMrTHM3d1dTakxqVE96N3M3MlZUSU5yYmx3cnRIRUlvM1JWTkV6SkNHYW5LSXdZMWZVSUtrCldNMlR0SDl5cnFsSGN0Z2pIUlFmRkVTRmlYaHJiODhSQmd0amIva0xtVGpCaTFBeEVadWNobWZ2QVRmNENOY3EKY21QcHNqdDBPTnI0YnhJMVRyUWxEemNiN1hMSFBrWW91SUprdnVzMUZvaTEySmRNM1MrK3labFZPMUZmZkU3bwpKMjhUdGJoZ3JGOGtIU0NMSkJvTTFSZ3FPZG9OVm5QOC9EOWZhamNNN0lWd2V4a0lSOTNKR1FJREFRQUJvMU13ClVUQWRCZ05WSFE0RUZnUVVlQ3JZanFJQzc1VUpxVmZEMDh1ZWdqeDZiUmN3SHdZRFZSMGpCQmd3Rm9BVWVDclkKanFJQzc1VUpxVmZEMDh1ZWdqeDZiUmN3RHdZRFZSMFRBUUgvQkFVd0F3RUIvekFOQmdrcWhraUc5dzBCQVFzRgpBQU9DQVFFQUkyRTBUOUt3MlN3RjJ2MXBqaHV4M0lkWWV2SGFVSkRMb0tPd0hSRnFSOHgxZ2dRcGVEcFBnMk5SCkx3R08yek85SWZUMmhLaWd1d2orWnJ5SGxxcHlxQ0pLOHJEU28xZUVPekIyWmE2S1YrQTVscEttMWdjV3VHYzMKK1UrVTFzTDdlUjd3ZFFuVjU0TVU4aERvNi9sVHRMRVB2Mnc3VlNPSlFDK013ejgrTFJMdjVHSW5BNlJySWNhKwozM0wxNnB4ZEttd1pLYThWcnBnMXJ3QzRnY3dlYUhYMUNEWE42K0JIbzhvWG5YWkh6UG91cldYS1BoaGdXZ2J5CkNDcUdIK0NWNnQ1eFg3b05NS3VNSUNqRVZndnNLWnRqeTQ5VW5iNVZZbHQ0b1J3dTFlbGdzRDNjekltbjlLREQKNHB1REFvYTZyMktZZE4xVkxuN3F3VG1TbDlTU05RPT0KLS0tLS1FTkQgQ0VSVElGSUNBVEUtLS0tLQo=""")
f.close()

DB2_Hackathon_dsn = 'DATABASE={};HOSTNAME={};PORT={};PROTOCOL=TCPIP;UID={uid};PWD={pwd};SECURITY=SSL;'.format(
    'bludb',
    '44d15047-5d7e-4642-875b-ca7710cd82f5.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud',
    30859,
    uid='ab1b160b',
    pwd="""EcMyLMrhTfJqrIFJ""",
    cert=DB2_Hackathon_cert
)

### Creadenciais CSV

In [4]:
if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_7efe33a944e54231997eacfdaad94b04 = 'https://s3-api.us-geo.objectstorage.softlayer.net'
else:
    endpoint_7efe33a944e54231997eacfdaad94b04 = 'https://s3-api.us-geo.objectstorage.service.networklayer.com'

credentials = {
    'endpoint': endpoint_7efe33a944e54231997eacfdaad94b04,
    'service_id': 'iam-ServiceId-8c64e377-89e3-424a-b706-72ea9ab5fd59',
    'iam_service_endpoint': 'https://iam.cloud.ibm.com/oidc/token',
    'api_key': 'vNrUHeW9-R1HdOM2ld5PDpR96H0VdqryNzcmyVlNFne_'
}

configuration_name = 'os_7efe33a944e54231997eacfdaad94b04_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

spark = SparkSession.builder.getOrCreate()

### Tratamento DataFrame e Carregando no DB2

In [5]:
skucategoria = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('compraentregasku.csv', 'projetoviagrupo1-donotdelete-pr-q8ymlbqhihqlvv'))

Converter DataFrame Spark para Pandas

In [6]:
df = skucategoria.toPandas()

Variavel de conexão com o server DB2

In [7]:
con = ibm_db.connect(DB2_Hackathon_dsn,"","")

Número de colunas

In [8]:
len(df.columns)

35

Nome colunas

In [9]:
df.columns

Index(['idcompraentregasku', 'idcompraentregaskupai', 'iddescontocomprejunto',
       'iddescontobrindecausa', 'idcompraentrega', 'idsku',
       'idestoqueprovavel', 'valorvendaunidadesemdesconto',
       'valorvendaunidade', 'valorcupomnominal',
       'valorvendaunidademenoscupomnominal', 'valorfretecomdesconto',
       'valorfrete', 'flagestoqueimpacto', 'presentemensagem', 'presentede',
       'presentepara', 'idskukitsige', 'tipocombosige', 'datacriacaoregistro',
       'valorbogof', 'parceirorecomendacao', 'flagajusteoperacional',
       'valorfreteoriginalnaorentavel', 'statustdca', 'valorst', 'sequencial',
       'prazogarantiafornecedor', 'tipoestoque', 'valorjurosunitario',
       'idskureferencia', 'precoiofvalorvenda', 'valorcomissao',
       'percentualComissao', 'idbandeira'],
      dtype='object')

Query para inserção dos dados

In [10]:
sql = "INSERT INTO HACKATHON.STAGECOMPRAENTREGASKU VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"

Preparação da query e variavel de conexão

In [11]:
stmt = ibm_db.prepare(con, sql)

Gerando novo DataFrame

In [12]:
subset = df[['idcompraentregasku', 'idcompraentregaskupai', 'iddescontocomprejunto',
       'iddescontobrindecausa', 'idcompraentrega', 'idsku',
       'idestoqueprovavel', 'valorvendaunidadesemdesconto',
       'valorvendaunidade', 'valorcupomnominal',
       'valorvendaunidademenoscupomnominal', 'valorfretecomdesconto',
       'valorfrete', 'flagestoqueimpacto', 'presentemensagem', 'presentede',
       'presentepara', 'idskukitsige', 'tipocombosige', 'datacriacaoregistro',
       'valorbogof', 'parceirorecomendacao', 'flagajusteoperacional',
       'valorfreteoriginalnaorentavel', 'statustdca', 'valorst', 'sequencial',
       'prazogarantiafornecedor', 'tipoestoque', 'valorjurosunitario',
       'idskureferencia', 'precoiofvalorvenda', 'valorcomissao',
       'percentualComissao', 'idbandeira']]

Convertendo os registros para o tipo tuple

In [13]:
tuple_of_tuples = tuple([tuple(x) for x in subset.values])

Executando a query com os dados da variavel "tuple_of_tuples"

In [14]:
ibm_db.execute_many(stmt, tuple_of_tuples)

Exception: 